# Identifying High-Response Customer Segments

- **Objective**: Identify customer groups most likely to respond positively to marketing promotions.
- **Business Impact**: Helps target promotions efficiently, saving cost and improving ROI.

## Data Preparation
- Load the Dataset
- Inspect the data; column types, missing values, preview
- Handle missing values
- Clean & normalize values; standardize text fields, date conversion, duplicates
- Feature engineering; is_discounted, used_promo, loyalty_score, high_spender etc
- Encode categorical variables; OHE, label encoding(ordinal)
- Scale numerical features

### 1. Load & Inspect Data

In [271]:
# Import libraries
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
# sns.set_palette("pastel")
import matplotlib.ticker as ticker

In [272]:
# Load the dataset
df = pd.read_csv("data/shopping_behavior_updated.csv")
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [273]:
# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [274]:
# Summary stats
df.describe(include='all')

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
count,3900.000000,3900.000000,3900,3900,3900,3900.000000,3900,3900,3900,3900,3900.000000,3900,3900,3900,3900,3900.000000,3900,3900
unique,NaN,NaN,2,25,4,NaN,50,4,25,4,NaN,2,6,2,2,NaN,6,7
top,NaN,NaN,Male,Blouse,Clothing,NaN,Montana,M,Olive,Spring,NaN,No,Free Shipping,No,No,NaN,PayPal,Every 3 Months
freq,NaN,NaN,2652,171,1737,NaN,96,1755,177,999,NaN,2847,675,2223,2223,NaN,677,584
mean,1950.500000,44.068462,NaN,NaN,NaN,59.764359,NaN,NaN,NaN,NaN,3.749949,NaN,NaN,NaN,NaN,25.351538,NaN,NaN
std,1125.977353,15.207589,NaN,NaN,NaN,23.685392,NaN,NaN,NaN,NaN,0.716223,NaN,NaN,NaN,NaN,14.447125,NaN,NaN
min,1.000000,18.000000,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,975.750000,31.000000,NaN,NaN,NaN,39.000000,NaN,NaN,NaN,NaN,3.100000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN
50%,1950.500000,44.000000,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,3.700000,NaN,NaN,NaN,NaN,25.000000,NaN,NaN
75%,2925.250000,57.000000,NaN,NaN,NaN,81.000000,NaN,NaN,NaN,NaN,4.400000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN


### 2. Clean & Normalize Columns

In [275]:
# Standardize text columns and replace spaces with underscores
def standardize_text_columns(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

df = standardize_text_columns(df)

# Number of duplicate rows
print("Duplicates:", df.duplicated().sum())

Duplicates: 0


In [276]:
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount_(usd),location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


### 3. Feature Engineering

In this step we'll engineer some features that will help us distinguish who’s likely to respond to promotions. We’ll add four new columns:

- `Is_Discounted` – mirror of discount_applied (already 0/1) for clarity
- `Used_Promo` – mirror of promo_code_used (0/1)
- `Loyalty_Score` – combines Previous Purchases and Frequency of Purchases into one metric
- `High_Spender` – flags customers whose average Purchase Amount is above the 75th percentile

In [277]:
# Convert boolean columns to integers
bool_map = {'yes': 1, 'no': 0}
df['subscription_status'] = df['subscription_status'].astype(str).str.lower().map(bool_map)
df['discount_applied'] = df['discount_applied'].astype(str).str.lower().map(bool_map)
df['promo_code_used'] = df['promo_code_used'].astype(str).str.lower().map(bool_map)

df['frequency_of_purchases'] = (df['frequency_of_purchases'].astype(str).str.lower().str.strip())

# Map frequency_of_purchases → true numeric “purchases per year”
freq_map = {
    'annually': 1,
    'every 3 months': 4,
    'quarterly': 4,
    'monthly': 12,
    'bi-weekly': 26,
    'fortnightly': 26,
    'weekly': 52
}
df['freq_per_year'] = df['frequency_of_purchases'].map(freq_map)


# 3) Compute a loyalty_score on numeric bases:
#    - normalize previous_purchases and freq_per_year to [0,1]
#    - sum them so score ∈ [0,2]
scaler = MinMaxScaler()
df[['prev_norm','freq_norm']] = scaler.fit_transform(df[['previous_purchases','freq_per_year']])
df['loyalty_score'] = (df['prev_norm'] + df['freq_norm']).round(2)

# 4) Flag high spenders: those above the 75th percentile of purchase_amount_usd
threshold = df['purchase_amount_(usd)'].quantile(0.75)
df['high_spender'] = (df['purchase_amount_(usd)'] > threshold).astype(int)

# 5) Clean up intermediate columns
df.drop(columns=['prev_norm','freq_norm'], inplace=True)

In [278]:
df.sample(10, random_state=42)

,customer_id,age,gender,item_purchased,category,purchase_amount_(usd),location,size,color,season,...,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,freq_per_year,loyalty_score,high_spender
839,840,48,Male,Shirt,Clothing,31,Illinois,L,Teal,Spring,...,1,Free Shipping,1,1,10,Debit Card,every 3 months,4,0.24,0
1717,1718,29,Male,Hoodie,Clothing,50,Hawaii,L,Orange,Summer,...,0,Express,0,0,16,Debit Card,bi-weekly,26,0.80,0
321,322,41,Male,Sneakers,Footwear,36,Vermont,L,Orange,Summer,...,1,Free Shipping,1,1,48,Debit Card,quarterly,4,1.02,0
3187,3188,67,Female,Blouse,Clothing,72,Texas,L,Indigo,Winter,...,0,Free Shipping,0,0,28,Debit Card,weekly,52,1.55,0
2269,2270,49,Male,Skirt,Clothing,38,Kentucky,M,Yellow,Fall,...,0,Store Pickup,0,0,27,Credit Card,annually,1,0.53,0
366,367,64,Male,Socks,Clothing,36,Nebraska,M,Turquoise,Spring,...,1,Store Pickup,1,1,37,Venmo,fortnightly,26,1.22,0
2644,2645,25,Male,Pants,Clothing,83,New Mexico,M,Turquoise,Spring,...,0,Standard,0,0,4,Bank Transfer,every 3 months,4,0.12,1
1749,1750,43,Male,Hoodie,Clothing,70,Wyoming,L,Pink,Fall,...,0,2-Day Shipping,0,0,36,Debit Card,every 3 months,4,0.77,0
3605,3606,68,Female,Jeans,Clothing,41,Alabama,M,Gold,Fall,...,0,Store Pickup,0,0,36,Bank Transfer,every 3 months,4,0.77,0
1096,1097,57,Male,Coat,Outerwear,46,New Hampshire,S,Beige,Fall,...,0,Standard,1,1,48,Credit Card,annually,1,0.96,0


In [279]:
df[df['loyalty_score'] == 0]

,customer_id,age,gender,item_purchased,category,purchase_amount_(usd),location,size,color,season,...,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,freq_per_year,loyalty_score,high_spender
1228,1229,55,Male,Jacket,Outerwear,55,Indiana,S,Yellow,Winter,...,0,Express,1,1,1,Cash,annually,1,0.0,0
1610,1611,20,Male,Socks,Clothing,31,Tennessee,L,Purple,Spring,...,0,2-Day Shipping,1,1,1,PayPal,annually,1,0.0,0
1690,1691,40,Male,Coat,Outerwear,30,Nevada,M,Black,Summer,...,0,Free Shipping,0,0,1,Cash,annually,1,0.0,0
1772,1773,58,Male,Shoes,Footwear,34,Hawaii,M,Gold,Summer,...,0,2-Day Shipping,0,0,1,Bank Transfer,annually,1,0.0,0
1842,1843,61,Male,Handbag,Accessories,78,Alabama,M,Black,Summer,...,0,Express,0,0,1,Debit Card,annually,1,0.0,0
2174,2175,42,Male,Backpack,Accessories,49,Colorado,S,Olive,Spring,...,0,Store Pickup,0,0,1,Cash,annually,1,0.0,0
2471,2472,30,Male,Sandals,Footwear,83,Oklahoma,M,Gray,Spring,...,0,Store Pickup,0,0,1,Cash,annually,1,0.0,1
2573,2574,54,Male,T-shirt,Clothing,90,New Mexico,M,Red,Summer,...,0,Store Pickup,0,0,1,Bank Transfer,annually,1,0.0,1
3728,3729,62,Female,Jacket,Outerwear,31,Vermont,L,Red,Winter,...,0,Standard,0,0,1,Debit Card,annually,1,0.0,0


In [280]:
# # Filter numeric and categorical columns
# numeric_columns = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
# categorical_columns = [col for col in df.columns if pd.api.types.is_object_dtype(df[col])]

# # Combine numeric and categorical columns
# columns_to_plot = numeric_columns + categorical_columns

# # Set the number of rows and columns for the grid
# n_cols = 7  # Increase the number of columns for a landscape view
# n_rows = -(-len(columns_to_plot) // n_cols)  # Calculate rows to fit all plots

# # Create subplots
# fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
# axes = axes.flatten()  # Flatten the 2D array of axes for easy indexing

# # Plot each column
# for i, column in enumerate(columns_to_plot):
#     ax = axes[i]
#     if column in numeric_columns:
#         sns.histplot(df[column], kde=True, bins=30, ax=ax)
#         ax.set_title(f'Distribution of {column} (Numerical)')
#     elif column in categorical_columns:
#         sns.countplot(x=df[column], ax=ax, order=df[column].value_counts().index)
#         ax.set_title(f'Distribution of {column} (Categorical)')
#     ax.set_xlabel(column)
#     ax.set_ylabel('Frequency')

# # Remove unused subplots
# for j in range(len(columns_to_plot), len(axes)):
#     fig.delaxes(axes[j])  # Delete any unused axes

# plt.tight_layout()
# plt.show()

### Encode & Scale

- Check for Cardinality of Categorical Columns

In [281]:
# Check cardinality of categorical columns
categorical_cols = [
    'gender','item_purchased','category','location',
    'size','color','season','shipping_type','payment_method'
]

for col in categorical_cols:
    n_unique = df[col].nunique()
    print(f"{col}: {n_unique} unique values")

gender: 2 unique values
item_purchased: 25 unique values
category: 4 unique values
location: 50 unique values
size: 4 unique values
color: 25 unique values
season: 4 unique values
shipping_type: 6 unique values
payment_method: 6 unique values


From the cardinality check, here's what stands out:

- `location`: 50 unique values – high
- `item_purchased`: 25 unique values – moderate
- `color`: 25 unique values – moderate

We'll be reducing this counts to avoid bloated one-hot encoding

In [282]:
# Helper function to reduce categories
def reduce_categories(series, threshold=None):
    """
    Groups rare categories in a Series into 'Other'.
    threshold: proportion (e.g. 0.02 = 2%) below which to group.
    """
    freq = series.value_counts(normalize=True)
    keep = freq[freq > threshold].index
    return series.apply(lambda x: x if x in keep else 'Other')


df['location'] = reduce_categories(df['location'], threshold=0.02)
df['item_purchased'] = reduce_categories(df['item_purchased'], threshold=0.04)
df['color'] = reduce_categories(df['color'], threshold=0.04)

In [283]:
print(df[['location', 'item_purchased', 'color']].nunique())


location          23
item_purchased    16
color             11
dtype: int64


- Encoding Categorical Variables

In [284]:
categorical_cols = ['gender', 'item_purchased', 'category', 'location', 'size', 
                    'color', 'season', 'shipping_type', 'payment_method']

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(df_encoded.shape)
df_encoded.head()

(3900, 79)


,customer_id,age,purchase_amount_(usd),review_rating,subscription_status,discount_applied,promo_code_used,previous_purchases,frequency_of_purchases,freq_per_year,...,shipping_type_Express,shipping_type_Free Shipping,shipping_type_Next Day Air,shipping_type_Standard,shipping_type_Store Pickup,payment_method_Cash,payment_method_Credit Card,payment_method_Debit Card,payment_method_PayPal,payment_method_Venmo
0,1,55,53,3.1,1,1,1,14,fortnightly,26,...,1,0,0,0,0,0,0,0,0,1
1,2,19,64,3.1,1,1,1,2,fortnightly,26,...,1,0,0,0,0,1,0,0,0,0
2,3,50,73,3.1,1,1,1,23,weekly,52,...,0,1,0,0,0,0,1,0,0,0
3,4,21,90,3.5,1,1,1,49,weekly,52,...,0,0,1,0,0,0,0,0,1,0
4,5,45,49,2.7,1,1,1,31,annually,1,...,0,1,0,0,0,0,0,0,1,0


### Modeling

- Create a Binary Target

We'll use a composite taget to represent any positive reaction to a promotion

In [285]:
# 1. Define composite target
df_encoded['responded_to_promo'] = ((df_encoded['promo_code_used'] == 1) | (df_encoded['discount_applied'] == 1)).astype(int)

# 2. Inspect class balance
print(df_encoded['responded_to_promo'].value_counts(normalize=True))

0    0.57
1    0.43
Name: responded_to_promo, dtype: float64


In [286]:
# correlation of all numeric features with the target
corrs = df_encoded.corr()['responded_to_promo'].abs().sort_values(ascending=False)
print(corrs.head(15))

responded_to_promo            1.000000
discount_applied              1.000000
promo_code_used               1.000000
customer_id                   0.857496
subscription_status           0.700202
gender_Male                   0.595823
item_purchased_Socks          0.042873
location_Indiana              0.040551
item_purchased_Blouse         0.039283
item_purchased_Other          0.025791
item_purchased_Sandals        0.025589
item_purchased_Coat           0.025435
previous_purchases            0.023537
shipping_type_Next Day Air    0.022764
size_S                        0.021937
Name: responded_to_promo, dtype: float64


- Train/Test Split & Scaling

In [287]:
# 1. Drop leakage & ID columns from df_encoded
leakage_and_id = ['promo_code_used','discount_applied','customer_id', 'frequency_of_purchases']
df_model = df_encoded.drop(columns=leakage_and_id + ['responded_to_promo'], errors='ignore')

# 2. Define X and y
X = df_model.copy()
y = df_encoded['responded_to_promo']

# 3. Train‑test split (stratified)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 4. Scale numeric features
from sklearn.preprocessing import StandardScaler
num_cols = ['age', 'previous_purchases', 'freq_per_year', 'loyalty_score']
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

1. Logistic Regression - Baseline

In [288]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# 1. Instantiate & train
lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(X_train, y_train)

# 2. Predict & evaluate
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:,1]

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))

print("Logistic Regression ROC AUC:", roc_auc_score(y_test, y_proba_lr))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       445
           1       0.83      0.75      0.79       335

    accuracy                           0.83       780
   macro avg       0.83      0.82      0.82       780
weighted avg       0.83      0.83      0.82       780

Logistic Regression ROC AUC: 0.9205634747610263


2. Random Forest

In [289]:
from sklearn.ensemble import RandomForestClassifier

# 1. Instantiate & train
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    class_weight='balanced',
    random_state=42
)
rf.fit(X_train, y_train)

# 2. Predict & evaluate
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

print("Random Forest ROC AUC:", roc_auc_score(y_test, y_proba_rf))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.89      0.84       445
           1       0.82      0.70      0.76       335

    accuracy                           0.81       780
   macro avg       0.81      0.79      0.80       780
weighted avg       0.81      0.81      0.80       780

Random Forest ROC AUC: 0.9131913466375985


- `Logistic Regression` edges out `Random Forest` by a small margin (0.9206 vs. 0.9132 AUC).
- It’s also more interpretable: you can directly inspect coefficients to see which features drive response.
- We'll use Logistic Regression as our primary model, and keep Random Forest as a backup or for further comparison after tuning.

- **Inspect Logistic Regression Coefficients**

In [294]:
import numpy as np
feature_names = X_train.columns
coefs = lr.coef_[0]

# Create a sorted list of (feature, coefficient) by absolute value
imp = sorted(zip(feature_names, coefs), key=lambda x: abs(x[1]), reverse=True)[:10]
print("\nTop 10 Features from Logistic Regression:")
for feat, weight in imp:
    print(f"{feat:30}  {weight:.3f}")


Top 10 Features from Logistic Regression:
subscription_status             5.484
gender_Male                     5.008
location_Indiana                0.924
location_Mississippi            0.592
item_purchased_Socks            -0.548
location_Arkansas               0.510
location_Maryland               0.466
item_purchased_Sandals          -0.444
color_Green                     -0.420
item_purchased_Skirt            -0.371


- **Examine Random Forest Feature Importances**

In [295]:
importances = rf.feature_importances_
imp_rf = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 Features from Random Forest:")
for feat, imp_val in imp_rf:
    print(f"{feat:30}  {imp_val:.3f}")


Top 10 Features from Random Forest:
subscription_status             0.416
gender_Male                     0.378
previous_purchases              0.020
loyalty_score                   0.017
purchase_amount_(usd)           0.016
review_rating                   0.014
age                             0.013
freq_per_year                   0.007
item_purchased_Blouse           0.006
item_purchased_Socks            0.005


### Hyperparameter Tuning via GridSearchCV

- **Logistic Regression: tune regularization strength C**

In [296]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression

# Set up parameter grid
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],       # inverse regularization strength
    'penalty': ['l1','l2'],            # L1 or L2 regularization
    'solver': ['liblinear']            # supports L1
}

# 5‑fold stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_lr = GridSearchCV(
    LogisticRegression(class_weight='balanced', random_state=42),
    param_grid_lr,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)
grid_lr.fit(X_train, y_train)

print("Best LR params:", grid_lr.best_params_)
print("Best LR CV AUC:", grid_lr.best_score_)

Best LR params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best LR CV AUC: 0.919899371286116


- **Random Forest: tune `n_estimators`, `max_depth`, `min_samples_leaf`**

In [297]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, None],
    'min_samples_leaf': [1, 5, 10]
}

grid_rf = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=42),
    param_grid_rf,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)
grid_rf.fit(X_train, y_train)

print("Best RF params:", grid_rf.best_params_)
print("Best RF CV AUC:", grid_rf.best_score_)

Best RF params: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 100}
Best RF CV AUC: 0.9257110322436463


### Cross-Validation Stability

When we tuned our models with GridSearchCV, we optimized hyperparameters based on the average AUC across folds, but a single 'best score' doesn't tell us how much that performance might vary if the data were split differently. 

Let's go ahead and perform cross-validation to check if our model achieves consistent AUC across multiple folds.

In [298]:
from sklearn.model_selection import cross_val_score

best_lr = grid_lr.best_estimator_
best_rf = grid_rf.best_estimator_

lr_scores = cross_val_score(best_lr, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
rf_scores = cross_val_score(best_rf, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)

print("LR 5‑fold AUC:", lr_scores, "mean:", lr_scores.mean())
print("RF 5‑fold AUC:", rf_scores, "mean:", rf_scores.mean())

LR 5‑fold AUC: [0.92234404 0.91393803 0.92753752 0.91403738 0.92163988] mean: 0.919899371286116
RF 5‑fold AUC: [0.92901015 0.91673654 0.92867998 0.9271166  0.92701189] mean: 0.9257110322436463


### Model Selection

| Model               | 5‑Fold CV AUC | Notes                                         |
| ------------------- | ------------- | --------------------------------------------- |
| Logistic Regression |        0.9199 | Very interpretable, nearly as good            |
| Random Forest       |        0.9257 | Slightly better AUC, captures non‑linearities |

The model performs consistently across different folds, the stable performance suggests the model will generalize better to truly unseen data (future customers).

> **Confidence in Business Decisions**

When presenting to stakeholders, you can say:

“Our model’s AUC ranged only ±0.005 across 5 folds—so we expect similarly high performance in production.”

- We'll use the `Random Forest` as you primary scoring model, and retain the Logistic Regression as an interpretable check.

Best model report:

```
Mean AUC: 0.9257 (matches our GridSearchCV best score)

Range: 0.917–0.929 → a spread of ~0.012

Std. Dev.: small (≈0.005), indicating high stability

That tells us the model’s performance is consistently high, and we can trust its AUC estimate.
```


>Side note:
In practice, always report both the mean and the standard deviation of CV scores.
If the std. dev. is large (>0.02–0.03), consider gathering more data, simplifying the model, or adding regularization.
Use these stability metrics when discussing model readiness with stakeholders.

### Threshold Selection

In [299]:
import numpy as np
from sklearn.metrics import f1_score, roc_curve

# get LR probabilities on validation fold
y_proba = best_lr.predict_proba(X_test)[:,1]

# Evaluate F1 at various thresholds
thresholds = np.linspace(0.1, 0.9, 81)
f1s = [f1_score(y_test, y_proba >= t) for t in thresholds]
best_idx = np.argmax(f1s)
best_thresh, best_f1 = thresholds[best_idx], f1s[best_idx]
print(f"Best threshold by F1: {best_thresh:.2f} → F1 = {best_f1:.3f}")

# Youden’s J (sensitivity + specificity −1)
fpr, tpr, thr = roc_curve(y_test, y_proba)
youden = tpr - fpr
j_idx = np.argmax(youden)
youden_thresh = thr[j_idx]
print(f"Youden’s J threshold: {youden_thresh:.2f}")

Best threshold by F1: 0.46 → F1 = 0.794
Youden’s J threshold: 0.46


Optimal probability cutoff at 0.46 (both by F1 and Youden’s J); customers with predicted ≥ 0.46 are “likely responders.”

### Business Validation (Uplift Simulation)

In [301]:
# Prepare simulation DataFrame
df_sim = X_test.copy()
df_sim['true']      = y_test.values
df_sim['proba']     = best_lr.predict_proba(X_test)[:,1]
df_sim['purchase']  = df_encoded.loc[df_sim.index, 'purchase_amount_(usd)']

# Target top X% by proba or use threshold
cutoff = best_thresh
df_target = df_sim[df_sim['proba'] >= cutoff]

# Compute metrics
n = len(df_target)
resp_rate = df_target['true'].mean()
baseline = df_sim['true'].mean()
avg_purchase = df_target['purchase'].mean()

rev_target = n * resp_rate * avg_purchase
rev_base   = n * baseline * avg_purchase
uplift_rev = rev_target - rev_base

print(f"Target size: {n}")
print(f"Resp rate targeted: {resp_rate:.2%}, baseline: {baseline:.2%}")
print(f"Avg purchase: ${avg_purchase:.2f}")
print(f"Revenue targeted: ${rev_target:,.0f}")
print(f"Baseline revenue: ${rev_base:,.0f}")
print(f"Estimated uplift: ${uplift_rev:,.0f}")


Target size: 343
Resp rate targeted: 78.43%, baseline: 42.95%
Avg purchase: $55.85
Revenue targeted: $15,023
Baseline revenue: $8,227
Estimated uplift: $6,796


| Metric                      |        Value |
| --------------------------- | -----------: |
| Number Targeted             |          343 |
| Response Rate (targeted)    |      78.43 % |
| Baseline Response Rate      |      42.95 % |
| Average Purchase per Target |      \$55.85 |
| Revenue with Targeting      |     \$15,023 |
| Revenue Baseline            |     \$ 8,227 |
| **Incremental Uplift**      | **\$ 6,796** |


By focusing your promotion on the top 44 % of customers by model score (those ≥ 0.46), you would have generated an additional $6.8 K over this test slice, versus a non‑targeted campaign.

**Extrapolation to Full Population**

Our test set was 20 % of the 3,900 customers (≈ 780). If the same uplift holds across all customers:

Total Uplift ≈ $6,796 / 0.20 = $33,980

So, a one‑time campaign across all 3,900 customers could yield ~$34 K in extra revenue.

---

**Business Recommendations**

Deploy the RF model in your campaign system to score all customers.

Target those with score ≥ 0.46—this group is most likely to redeem offers.

Monitor real‑time results against the model’s predictions to validate uplift.

A/B test: Compare this targeted approach vs. a random‑control group to measure actual lift

**Next Analytical Steps**

Cost–Benefit Analysis: Compare promo cost (discounts) to incremental revenue to compute ROI.

Refine Segments: Drill into subgroups (e.g., non‑subscribers in Indiana) for micro‑targeting.

Time‑Series Validation: Run the model on a subsequent campaign period to confirm stability.

Dashboarding: Build a dashboard to track predicted vs. actual response and revenue in near real‑time.